## File search assistant
The McKessonMDTranscripts demonstrates the creation and execution of an assistant using the file_search feature with a knowledge base based on earnings call transcripts downloaded from the internet.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [6]:
#r "nuget: AntRunnerLib, 0.7.11"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.7.11

## Ensure the McKessonMDTranscripts assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\McKessonMDTranscripts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"McKessonMDTranscripts" contains the following files:
```
│   manifest.json
│   prompt.md
│
└───VectorStores
    └───McKessonMDTranscripts
            211101-MCK-Q2FY22-Earnings-Call-Transcript.md
            220202-MCK-Q3FY22-Earnings-Call-Transcript.md
            220505-MCK-Q4FY22-Earnings-Call-Transcript.md
            220803-MCK-Q1FY23-Earnings-Call-Transcript.md
            230508-MCK-Q4FY23-Earnings-Call-Transcript.md
            230802-MCK-Q1FY24-Earnings-Call-Transcript.md
            231101-MCK-Q2FY24-Earnings-Call-Transcript.md
            MCK-Q1FY21-Transcript.md
            MCK-Q1FY22-Transcript.md
            MCK-Q2-FY23-Earnings-Transcript.md
            MCK-Q2FY21-Transcript.md
            MCK-Q3-FY24-Earnings-Transcript.md
            MCK-Q4-FY24-Earnings-Call-Transcript.md
            MCK-Q4FY21-Transcript.md
            MCK-US-20230201-2761065-C.md
            Q1-FY19-Earnings-Call-Transcript.md
            Q1-FY20-Earnings-Call-Transcript.md
            Q2-FY19-Earnings-Call-Transcript.md
            Q2-FY20-Earnings-Call-Transcript.md
            Q3-FY19-Earnings-Call-Transcript.md
            Q3FY20-MCK-Earnings-Call-Transcript.md
            Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
            Q4-FY19-Earnings-Call-Transcript.md
            Q4FY20-MCK-Corrected-Transcript.md
```

### Explanation
`AssistantUtility.Create` will create the vector store if necessary, upload the files, and then update the manifest with the vector store ID before creating the assistant.

In [7]:
var assistantId = await AssistantUtility.Create("McKessonMDTranscripts", config);
Console.WriteLine(assistantId)

asst_N88ilPklyMFpVEbG2cMnlWBU


## Run the Assistant

`output.Dialog` shows the conversation.

In [8]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "McKessonMDTranscripts",
    Instructions = "How did covid impact results over time?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: How did covid impact results over time?

Assistant: I searched my knowledge base for the answer.

The COVID-19 pandemic significantly impacted McKesson's results over time. Here are the key points illustrating the impact:

1. **Initial Impact and Revenue Increases**:
   - At the onset of the pandemic, McKesson saw increased demand for pharmaceuticals and personal protective equipment (PPE) as people stocked up on medications and supplies. This led to an estimated $2 billion of incremental revenue in Q4 FY20【4:1†source】【4:13†source】.

2. **Fluctuating Demand and Volume Declines**:
   - In FY21, McKesson experienced volume declines initially due to the impact of COVID-19 but saw improvements as the year progressed. The company's fiscal fourth quarter saw continued volatility from COVID-19, a weak cold and flu season, and a winter storm that impacted parts of the U.S. Pharmaceutical business【4:4†source】.

3. **Vaccine Distribution and Testing**:
   - The distribution of COVID-19 vaccines and related supplies provided substantial benefits. For example, in Q4 FY21, McKesson recognized benefits from distributing COVID-19 vaccines and ancillary supply kits【4:4†source】【4:10†source】.
   - However, as the pandemic evolved, the demand for COVID-19-related products and services fluctuated. In FY22, McKesson noted a 60% increase in COVID-19 test demand in January due to the Omicron variant, followed by a significant decline in February and March【4:5†source】.

4. **Profit and Earnings Impact**:
   - The impact of COVID-19 on earnings varied over time. In Q4 FY20, the pandemic had a nominal effect on adjusted earnings as increased revenue was offset by higher operating expenses, such as investments in frontline employees and remote workforce support【4:13†source】.
   - In Q4 FY21, adjusted earnings per diluted share increased by 18% compared to the prior year, driven by contributions from COVID-19 vaccine distribution programs【4:4†source】【4:10†source】.

5. **Reduced Contribution from COVID-19 Programs**:
   - By FY23, the contribution from COVID-19 programs had decreased. For instance, the Medical-Surgical Solutions segment saw lower sales of COVID-19 tests and a reduced contribution from kitting, storage, and distribution of ancillary supplies compared to the prior year【4:7†source】.

Overall, McKesson's financial performance during the pandemic was marked by initial revenue boosts from increased demand for pharmaceuticals and PPE, followed by fluctuating demand for COVID-19-related products and services, and eventually a reduction in contributions from these programs as the pandemic's impact evolved.


In [9]:
output.LastMessage.DisplayAs("text/markdown");

The COVID-19 pandemic significantly impacted McKesson's results over time. Here are the key points illustrating the impact:

1. **Initial Impact and Revenue Increases**:
   - At the onset of the pandemic, McKesson saw increased demand for pharmaceuticals and personal protective equipment (PPE) as people stocked up on medications and supplies. This led to an estimated $2 billion of incremental revenue in Q4 FY20【4:1†source】【4:13†source】.

2. **Fluctuating Demand and Volume Declines**:
   - In FY21, McKesson experienced volume declines initially due to the impact of COVID-19 but saw improvements as the year progressed. The company's fiscal fourth quarter saw continued volatility from COVID-19, a weak cold and flu season, and a winter storm that impacted parts of the U.S. Pharmaceutical business【4:4†source】.

3. **Vaccine Distribution and Testing**:
   - The distribution of COVID-19 vaccines and related supplies provided substantial benefits. For example, in Q4 FY21, McKesson recognized benefits from distributing COVID-19 vaccines and ancillary supply kits【4:4†source】【4:10†source】.
   - However, as the pandemic evolved, the demand for COVID-19-related products and services fluctuated. In FY22, McKesson noted a 60% increase in COVID-19 test demand in January due to the Omicron variant, followed by a significant decline in February and March【4:5†source】.

4. **Profit and Earnings Impact**:
   - The impact of COVID-19 on earnings varied over time. In Q4 FY20, the pandemic had a nominal effect on adjusted earnings as increased revenue was offset by higher operating expenses, such as investments in frontline employees and remote workforce support【4:13†source】.
   - In Q4 FY21, adjusted earnings per diluted share increased by 18% compared to the prior year, driven by contributions from COVID-19 vaccine distribution programs【4:4†source】【4:10†source】.

5. **Reduced Contribution from COVID-19 Programs**:
   - By FY23, the contribution from COVID-19 programs had decreased. For instance, the Medical-Surgical Solutions segment saw lower sales of COVID-19 tests and a reduced contribution from kitting, storage, and distribution of ancillary supplies compared to the prior year【4:7†source】.

Overall, McKesson's financial performance during the pandemic was marked by initial revenue boosts from increased demand for pharmaceuticals and PPE, followed by fluctuating demand for COVID-19-related products and services, and eventually a reduction in contributions from these programs as the pandemic's impact evolved.

## Annotations

In [10]:
Console.WriteLine(JsonSerializer.Serialize(output.Annotations, new JsonSerializerOptions() { WriteIndented = true }));

[
  {
    "type": "file_citation",
    "text": "\u30104:1\u2020source\u3011",
    "start_index": 414,
    "end_index": 426,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:13\u2020source\u3011",
    "start_index": 426,
    "end_index": 439,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:4\u2020source\u3011",
    "start_index": 803,
    "end_index": 815,
    "file_citation": {
      "file_id": "assistant-mLYhwg7I9ywfWQPh1xEf1IXY",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:4\u2020source\u3011",
    "start_index": 1069,
    "end_index": 1081,
    "file_citation": {
      "file_id": "assistant-mLYhwg7I9ywfWQPh1xEf1IXY",
      "file_name": "",
      "quo

### Formatting Output
The current version of the API leaves some things to be desired. First, because of chunking it can cite the same source twice. Second, 'quote' is supposed to contain a snippet, but currently, it is always null

In [11]:
var files = new Dictionary<string, string>();
var annotatedMessage = output.LastMessage;
foreach(var annotation in output.Annotations)
{
    // Only get the file name once per id
    if(!files.ContainsKey(annotation.FileCitation.FileId))
    {
        files[annotation.FileCitation.FileId] = await annotation.GetFileName(config);
    }
    // But the annotation itself may be a duplicate citation to the same file, so one must process them all
    // You can format the replacement however you like. In this case it just makes an md link
    annotatedMessage = annotatedMessage.Replace(annotation.Text, $" [{files[annotation.FileCitation.FileId]}]({files[annotation.FileCitation.FileId]})");
}

foreach(var file in files)
{
    Console.WriteLine($"{file.Key} = {file.Value}");
}
Console.WriteLine("-------------------");
annotatedMessage.DisplayAs("text/markdown");

assistant-v1vgOsajg7zOoaHPJaHKvA3i = McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md
assistant-mLYhwg7I9ywfWQPh1xEf1IXY = McKessonMDTranscripts_MCK-Q4FY21-Transcript.md
assistant-KVLQgIZo6R8UzDv01PXDkrR6 = McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md
assistant-fslYRzYxsqiFwPGd7Fns83se = McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md
-------------------


The COVID-19 pandemic significantly impacted McKesson's results over time. Here are the key points illustrating the impact:

1. **Initial Impact and Revenue Increases**:
   - At the onset of the pandemic, McKesson saw increased demand for pharmaceuticals and personal protective equipment (PPE) as people stocked up on medications and supplies. This led to an estimated $2 billion of incremental revenue in Q4 FY20 [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md) [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).

2. **Fluctuating Demand and Volume Declines**:
   - In FY21, McKesson experienced volume declines initially due to the impact of COVID-19 but saw improvements as the year progressed. The company's fiscal fourth quarter saw continued volatility from COVID-19, a weak cold and flu season, and a winter storm that impacted parts of the U.S. Pharmaceutical business [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).

3. **Vaccine Distribution and Testing**:
   - The distribution of COVID-19 vaccines and related supplies provided substantial benefits. For example, in Q4 FY21, McKesson recognized benefits from distributing COVID-19 vaccines and ancillary supply kits [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).
   - However, as the pandemic evolved, the demand for COVID-19-related products and services fluctuated. In FY22, McKesson noted a 60% increase in COVID-19 test demand in January due to the Omicron variant, followed by a significant decline in February and March [McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md](McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md).

4. **Profit and Earnings Impact**:
   - The impact of COVID-19 on earnings varied over time. In Q4 FY20, the pandemic had a nominal effect on adjusted earnings as increased revenue was offset by higher operating expenses, such as investments in frontline employees and remote workforce support [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).
   - In Q4 FY21, adjusted earnings per diluted share increased by 18% compared to the prior year, driven by contributions from COVID-19 vaccine distribution programs [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).

5. **Reduced Contribution from COVID-19 Programs**:
   - By FY23, the contribution from COVID-19 programs had decreased. For instance, the Medical-Surgical Solutions segment saw lower sales of COVID-19 tests and a reduced contribution from kitting, storage, and distribution of ancillary supplies compared to the prior year [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md).

Overall, McKesson's financial performance during the pandemic was marked by initial revenue boosts from increased demand for pharmaceuticals and PPE, followed by fluctuating demand for COVID-19-related products and services, and eventually a reduction in contributions from these programs as the pandemic's impact evolved.

## Clean Up

In [12]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "McKessonMDTranscripts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
